In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os, glob
import json
import pandas as pd
import numpy as np
from PIL import Image
import shutil
from natsort import natsorted, ns

In [ ]:
pwd

### 이미지 폴더와 json 폴더의 목록 가져오기

In [ ]:
path = '/content/drive/MyDrive/yolov4/100_data'
data_folder = os.listdir(path)
json_folder = []
img_folder = []

for i in data_folder:
    if 'json' in i:
        json_folder.append(i)
    else :
        img_folder.append(i)
json_folder.sort()
img_folder.sort()

In [ ]:
len(img_folder), len(json_folder)

(100, 100)

In [ ]:
cd /content/drive/MyDrive/yolov4/

/content/drive/MyDrive/yolov4


#### yolov4 git 가져오기

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15308, done.
remote: Total 15308 (delta 0), reused 0 (delta 0), pack-reused 15308
Receiving objects: 100% (15308/15308), 13.69 MiB | 5.91 MiB/s, done.
Resolving deltas: 100% (10404/10404), done.
Checking out files: 100% (2044/2044), done.


In [ ]:
len(os.listdir('/content/drive/MyDrive/yolov4/annotation/train'))

16000

In [ ]:
len(os.listdir('/content/drive/MyDrive/yolov4/annotation/valid'))

4000

### annotation 파일 만들기
- 클래스인덱스 x y w h

In [ ]:
path = '/content/drive/MyDrive/yolov4/100_data/'
annotation_lst = []
saved_txt_dir = '/content/drive/MyDrive/yolov4/annotation/valid/'

for idx, json_folder_name in enumerate(json_folder): # [:100]이면 클래스 100개
    # print(json_folder_name[:-5])
    # 폴더 안에 모든 json 파일 가져오기
    json_file = glob.glob(os.path.join(path + json_folder_name, '*.json'))
    # print(path + json_folder_name)
    json_file.sort()
    json_file = json_file[160:] #train set 만들때는 [:80], test set 만들때는 [80:]

    # 한 json 폴더의 모든 json 파일에 적용
    for j in range(len(json_file)): # 100개
        with open(json_file[j]) as f: 
          print(idx, json_folder_name[:-5], j)
          json_data = json.load(f)
          # {} 개수만큼 반복 - 좌표가 2개 이상인 경우 모든 좌표를 하나의 txt파일에 넣기 위함
          for i in range(len(json_data)):
            point = json_data[i]["Point(x,y)"]
            x = float(point.split(',')[0])
            y = float(point.split(',')[1])
            w = float(json_data[i]["W"])
            h = float(json_data[i]["H"])

            annotation_value = str(idx) + ' ' + str(x) + ' ' + str(y) + ' ' + str(w) + ' ' + str(h)
            annotation_lst.append(annotation_value)
            fname = json_data[0]["Code Name"][:-4]+'.txt'
            np.savetxt(saved_txt_dir + fname, annotation_lst, delimiter=',', fmt='%s') # txt 파일로 저장
        annotation_lst = [] # 다음 json파일을 txt파일로 변환하기 위한 리스트 초기화


### 이미지 경로가 들어가있는 파일 만들기
- train.txt : 학습할 이미지들의 경로
- valid.txt : 검증할 이미지들의 경로

In [ ]:
image_dir = '/content/drive/MyDrive/yolov4/annotation/train/'
all_image = glob.glob(os.path.join(image_dir, '*jpg'))
len(all_image)
image_dir_list = np.array([])
image_dir_list = np.append(image_dir_list, all_image)
len(image_dir_list)

In [ ]:
np.savetxt('train.txt', image_dir_list, delimiter=',', fmt='%s') # txt 파일로 저장

In [ ]:
image_dir = '/content/drive/MyDrive/yolov4/annotation/valid/'
all_image = glob.glob(os.path.join(image_dir, '*jpg'))
len(all_image)
image_dir_list = np.array([])
image_dir_list = np.append(image_dir_list, all_image)
len(image_dir_list)
np.savetxt('valid.txt', image_dir_list, delimiter=',', fmt='%s') # txt 파일로 저장

### obj.data 파일 수정
#### - 학습정보가 담겨있음
- classes= 100
- train=/content/drive/MyDrive/yolov4/- train.txt
- valid=/content/drive/MyDrive/yolov4/valid.txt
- names=/content/drive/MyDrive/yolov4/obj.names
- backup=/content/drive/MyDrive/yolov4/training

### cfg 파일 수정
#### - 모델 구조 및 train 과 관련된 설정이 들어있는 파일
- darknet/cfg에 들어가서 사용할 cfg 파일 수정
- subdivisions: 미니배치수
- width, height는 32의 배수로 설정
- max_batches: epoch 수 (클래스개수 * 2000)
- steps: max_batches의 80%, 90%
- classes: 클래스 개수 ([yolo] 레이어 안에 있음, 모든 [yolo]레이어에 적용)
- filters: (4 + 1 + class 수) * 3 ( [yolo] 레이어 바로 위에 있는 [convolutional] 레이어의 filters만 수정!!, 모든 [yolo]레이어 위에 적용)
- anchors: 앵커 개수 구하는 코드를 통해 나온값 입력 (모든 anchors에 적용)


In [ ]:
cd ./darknet

/content/drive/My Drive/yolov4/darknet


In [ ]:
# !make실행할 경우 chmod +x *.sh 에러나면 아래 코드로 실행
!chmod +x ./darknet

chmod: cannot access 'darknet': No such file or directory


In [ ]:
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -c ./src/image_opencv.cpp -o obj/image_opencv.o
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -c ./src/http_stream.cpp -o obj/http_stream.o
./src/http_stream.cpp: In member function ‘bool JSON_sender::write(const char*)’:
./src/http_stream.cpp:253:21: warning: unused variable ‘n’ [-Wunused-variable]
                 int n = _write(client, outputbuf, outlen);
                     ^
./src/http_stream.cpp: In function ‘void set_track_id(detection*, int, float, float, float, int, int, int)’:
./src/http_stream.cpp:867:27: warning: comparison between signed and unsigned integer expressions [-Wsign-compare]
         for (int i = 0; i < v.size(); ++i) {
                         ~~^~~~~~~~~~
./src/http_stream.cpp:875:33: warning: c

In [ ]:
!./darknet detector calc_anchors /content/drive/MyDrive/yolov4/obj.data -num_of_clusters 9 -width 512 -height 512

 GPU isn't used 
 OpenCV isn't used - data augmentation will be slow 

 num_of_clusters = 9, width = 512, height = 512 
 read labels from 16000 images 
 loaded 	 image: 16000 	 box: 18037
 all loaded. 

 calculating k-means++ ...

 iterations = 71 


counters_per_class = 201, 161, 173, 160, 162, 160, 167, 166, 178, 244, 160, 169, 329, 171, 174, 164, 160, 170, 164, 227, 225, 166, 236, 160, 161, 166, 163, 160, 162, 162, 179, 163, 163, 162, 162, 163, 189, 165, 169, 208, 161, 182, 173, 265, 161, 166, 164, 160, 162, 161, 201, 160, 229, 174, 171, 162, 222, 161, 190, 213, 458, 161, 163, 266, 163, 184, 161, 161, 164, 163, 173, 170, 161, 165, 169, 161, 161, 164, 167, 162, 160, 300, 188, 166, 165, 164, 175, 183, 161, 201, 162, 172, 185, 162, 163, 160, 178, 186, 161, 166

 avg IoU = 80.55 % 

Saving anchors to the file: anchors.txt 
anchors = 101, 98, 152,187, 280,167, 205,304, 306,314, 427,226, 350,432, 472,334, 487,479
^C


### .names 파일 만들기
- class 이름 목록

In [ ]:
class_label = pd.DataFrame(img_folder)

class_label.to_csv('/content/drive/MyDrive/yolov4/obj.names', header=None, index=None)

In [ ]:
# make_image_dir_file(image_dir, 80, True, 'train')
make_image_dir_file(image_dir, 80, False, 'valid')

### train.txt, valid.txt를 만들기 위한 폴더 옮기는 작업

In [ ]:
# 이거 위로 !!
image_dir = '/content/drive/MyDrive/yolov4/images/train/'

# 이거보고 잘 편집해서 move_file 함수 바꾸삼 함수에 인자로 train, valid 다 할 수 있게끔1
def make_image_dir_file(image_dir, indexing_percentage, train, save_name):
    image_dir_list = np.array([])
    all_image = glob.glob(os.path.join(image_dir+folder_name, '*jpg'))

    for folder_name in img_folder:
        all_image = glob.glob(os.path.join(image_dir+folder_name, '*jpg'))

        image_len = len(all_image)
        train_test_split = int(image_len * indexing_percentage / 100)

        if train :
            all_image = all_image[:train_test_split]
        else :
            all_image = all_image[train_test_split:]

        print('class name:', folder_name, 'image 길이:', len(all_image))
        image_dir_list = np.append(image_dir_list, all_image)

    np.savetxt(save_name + '.txt', image_dir_list, delimiter=',', fmt='%s') # txt 파일로 저장

In [ ]:
# 해야됨 일단 이미지들을 train, valid로 분류한담에 train.txt valid.txt 만들어야됨
# 그이유는 일단 각 클래스별로 160, 40개씩 나누기 위함
# 일단 이미지들을 train, valid로 나누고 그걸 label.txt가 있는 곳으로 보내버려
import shutil
from natsort import natsorted, ns

def move_file(path, new_path, indexing_percentage, train, extention, overwrite=False): # overwrite = True -> 덮어씌우기모드
    dir = path + '/'
    print('원래 경로: ', dir)
    glob_file = os.listdir(dir)
    glob_file = natsorted(glob_file)
    print('길이: ', len(glob_file))
    new_dir = new_path + '/'
    print('새로운 경로: ', new_dir)
    image_len = len(all_image)
    train_test_split = int(image_len * indexing_percentage / 100)
    if train :
        glob_file = glob_file[:train_test_split]
    else :
        glob_file = glob_file[train_test_split:]
    

    for filename in glob_file:
        print('filename: ' + filename)
        print('원래 경로: ', dir+filename)
        print('새로운 경로:', new_dir+filename)
        try :
            print(new_dir+filename)

            shutil.copy(dir+filename, new_dir+filename)
        except shutil.Error:
            if overwrite == True:
                print("파일 덮어 씌우기")
                print(new_dir+filename)
                shutil.copy(dir+filename, new_dir+filename)
            else: 
                print("파일이 이미 존재함: " + new_dir + filename)

In [ ]:
# 이미지 옮기기
path = '/content/drive/MyDrive/yolov4/100_data/'
for i in img_folder:
  move_file(path = path+i, new_path= "/content/drive/MyDrive/yolov4/images/train/", 80, True, extention="jpg")
  move_file(path = path+i, new_path= "/content/drive/MyDrive/yolov4/images/valid/", 80, False, extention="jpg")

In [ ]:
# images_dir와 labels_dir 경로에 있는 파일들이 다 같은지 확인

images_dir = '/content/drive/MyDrive/yolov4/images/train/'
labels_dir = '/content/drive/MyDrive/yolov4/annotation/train/'
# labels_dir
images_file_list = os.listdir(images_dir)
images_file_list = natsorted(images_file_list)

for i, val in enumerate(images_file_list):
  images_file_list[i] = val.rstrip('.jpg')

labels_file_list = os.listdir(labels_dir)
labels_file_list = natsorted(labels_file_list)

for i, val in enumerate(labels_file_list):
  labels_file_list[i] = val.rstrip('.txt')

images_file_list = set(images_file_list)
labels_file_list = set(labels_file_list)
print(labels_file_list - images_file_list)
print(images_file_list - labels_file_list)

set()
set()
